https://adventofcode.com/2022/day/11

In [1]:
import re
import operator
from collections import deque
from functools import reduce
from math import lcm

In [2]:
with open("data/11.txt") as fh:
    data = fh.read()

In [3]:
def parse_monkey(s):
    L = s.strip().splitlines()
    return dict(
        id=int(re.search(r"\d+", L[0]).group()),
        items=tuple(int(x) for x in re.findall(r"\d+", L[1])),
        operation=L[2].split()[-2:],
        modulus=int(re.search(r"\d+", L[3]).group()),
        true_target=int(re.search(r"\d+", L[4]).group()),
        false_target=int(re.search(r"\d+", L[5]).group()),
    )

In [4]:
class Monkey:
    def __init__(self, id, items, operation, modulus, true_target, false_target, barrel=None):
        self.id = id
        self.items = deque(items)
        self.modulus = modulus
        self.true_target = true_target
        self.false_target = false_target
        self.barrel = barrel
        self.counter = 0

        self.opfunc = {"+": operator.add, "*": operator.mul}[operation[0]]
        try:
            self.oparg = int(operation[1])
        except ValueError:
            self.oparg = None

    def throw(self, item, target):
        self.barrel[target].catch(item)

    def catch(self, item):
        self.items.append(item)

    def manage_worry(self, item):
        return item // 3

    def op(self, item):
        arg = item if self.oparg is None else self.oparg
        return self.opfunc(item, arg)

    def turn(self):
        while self.items:
            self.counter += 1
            item = self.items.popleft()
            item = self.op(item)
            item = self.manage_worry(item)
            target = self.true_target if not item % self.modulus else self.false_target
            self.throw(item, target)

    def __repr__(self):
        return f"<Monkey {self.id} {self.counter} {list(self.items)}>"

In [5]:
%%time
barrel = [Monkey(**parse_monkey(s)) for s in data.split("\n\n")]
for monkey in barrel:
    monkey.barrel = barrel
for _ in range(20):
    for monkey in barrel:
        monkey.turn()
print(reduce(operator.mul, list(reversed(sorted(x.counter for x in barrel)))[:2]))

58322
CPU times: user 639 µs, sys: 117 µs, total: 756 µs
Wall time: 760 µs


In [6]:
class Monkey2(Monkey):
    def __init__(self, *args, **kwargs):
        self.barrel_modulus = 1
        super().__init__(*args, **kwargs)

    def manage_worry(self, item):
        return item % self.barrel_modulus

In [7]:
%%time
barrel = [Monkey2(**parse_monkey(s)) for s in data.split("\n\n")]
barrel_modulus = lcm(*[x.modulus for x in barrel])
for monkey in barrel:
    monkey.barrel = barrel
    monkey.barrel_modulus = barrel_modulus
for _ in range(10_000):
    for monkey in barrel:
        monkey.turn()
print(reduce(operator.mul, list(reversed(sorted(x.counter for x in barrel)))[:2]))

13937702909
CPU times: user 242 ms, sys: 0 ns, total: 242 ms
Wall time: 241 ms
